In [2]:
%cd /content
!git clone -b dev1 https://github.com/camenduru/PanoHead

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/ablation-trigridD-1-025000.pkl -d /content/PanoHead/models -o ablation-trigridD-1-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/baseline-easy-khair-025000.pkl -d /content/PanoHead/models -o baseline-easy-khair-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/easy-khair-180-gpc0.8-trans10-025000.pkl -d /content/PanoHead/models -o easy-khair-180-gpc0.8-trans10-025000.pkl

!pip install imgui glfw pyspng mrcfile ninja plyfile trimesh onnxruntime onnx

%cd /content
!git clone -b dev https://github.com/camenduru/3DDFA_V2
%cd /content/3DDFA_V2
!sh ./build.sh

!cp -rf /content/PanoHead/3DDFA_V2_cropping/test /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/dlib_kps.py /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/recrop_images.py /content/3DDFA_V2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/shape_predictor_68_face_landmarks/resolve/main/shape_predictor_68_face_landmarks.dat -d /content/3DDFA_V2 -o shape_predictor_68_face_landmarks.dat

!mkdir /content/in /content/stage /content/output

/content
Cloning into 'PanoHead'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 155 (delta 9), reused 6 (delta 6), pack-reused 145
Receiving objects: 100% (155/155), 29.40 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (45/45), done.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,475 kB of archives.
After this operation, 5,959 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0

In [3]:
%cd /content/3DDFA_V2
!rm -rf /content/stage/* /content/3DDFA_V2/crop_samples/img/* /content/3DDFA_V2/test/original/* /content/output/*
!cp /content/in/* /content/3DDFA_V2/test/original

!python dlib_kps.py
!python recrop_images.py -i data.pkl -j dataset.json

!cp -rf /content/3DDFA_V2/crop_samples/img/* /content/stage

# %cd /content/PanoHead
# !./gen_pti_script.sh

/content/3DDFA_V2
100% 1/1 [00:01<00:00,  1.68s/it]
results: 1


In [4]:
%cd /content/PanoHead
!python projector_withseg.py --shapes=True --save-video=False --num-steps=100 --num-steps-pti=100 --outdir=/content/output --target_img=/content/stage --network=/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --idx 0

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
projecting: [0] /content/stage/00667-20230704135111-527434926-models_03_realistic_majicmixRealistic_v6.webp
camera matrix: torch.Size([1, 25])
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %105 : int = prim::profile_ivalue(%103)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)
step:    1/100 mse: 0.43 perc: 0.64 w_norm: 0.00  noise: 0.00 
step:    2/100 mse: 0.55 perc: 0.62 w_norm: 0.00  noise: 0.00 
step:    3/100 mse: 0.44 perc: 0.62 w_norm: 0.00  noise: 0.00 
step:    4/100 mse: 0.17 perc: 0.56 w_norm: 0.00  noise: 0.00 
step:    5/100 mse: 0.06 perc: 0.

In [5]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/pre.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
100% 240/240 [00:25<00:00,  9.49it/s]


In [6]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/post.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
100% 240/240 [00:25<00:00,  9.52it/s]
